In [ ]:
from transformers import AutoTokenizer

tokenizer_base = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B")
tokenizer_base.save_pretrained("Llama-3.1-8B-Base")

In [ ]:
import json
tokenizer_json = json.load(open("Llama-3.1-8B-Base/tokenizer.json"))
vocab_base = tokenizer_json["model"]["vocab"]
merges_base = tokenizer_json["model"]["merges"]

In [ ]:
len(vocab_base)

In [ ]:
from collections import defaultdict
vocab_to_merges = defaultdict(list)
for k,v in vocab_base.items():
    if v < 256: continue
    print('------------------------------')
    for item in merges_base:
        left,right = item.split()
        if left+right == k:
            print('++',k,item)
            vocab_to_merges[k].append(item)

In [ ]:
import pickle as pkl
pkl.dump(vocab_to_merges, open("./Llama-3.1-8B-Base/vocab_to_merges_MAPPING.pkl", "wb"))

In [ ]:

tokenizer_json = json.load(open("Llama-3.1-8B-Base/tokenizer.json"))
vocab_base = tokenizer_json["model"]["vocab"]
merges_base = tokenizer_json["model"]["merges"]

In [ ]:
import os, glob
import pickle as pkl
import json
from collections import defaultdict

os.mkdir("EBM-Filter-FromScratch-Llama3_Vocab")

for fname in glob.glob("./EBM/*.txt"):
    print('***********Processing:',fname)
    vocab_to_merges = defaultdict(list) #pkl.load(open("./Llama-3.1-8B-Base/vocab_to_merges_MAPPING.pkl", "rb"))
    
    tokenizer_base = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B")
    tokenizer_json = json.load(open("Llama-3.1-8B-Base/tokenizer.json",'r',encoding='utf-8'))
    
    words_to_add = open(fname,'r',encoding='utf-8').read().splitlines()
    words_to_add = sorted(words_to_add, key=lambda x: len(x))

    for word in words_to_add:
        word = word.strip()
        split = tokenizer_base.tokenize(word if not word.startswith('Ġ') else ' '+word[1:])
        if len(split) == 1:
            continue
        
        if len(split) == 2: #pass
            vocab_to_merges[word] = [split[0],split[1]]
        
        if len(split) >= 3:
            # print('--word:',word,split)
            new_word = split[0]
            for i in range(1,len(split)):
                left = new_word
                right = split[i]
                new_word += split[i]
                # print('new_word:',new_word, 'Merge:',left,right)
                if new_word not in vocab_to_merges: vocab_to_merges[new_word] = [left,right]
    
    idx = 0
    for key,val in vocab_to_merges.items():
        if key not in tokenizer_json["model"]["vocab"]:
        # print(key,val,idx)
            print('Adding:',key,val)
            tokenizer_json["model"]["vocab"][key] = 128000+idx
            tokenizer_json["model"]["merges"].append(val)
            idx += 1
        
    tokenizer_json['post_processor']['processors'][-1]['special_tokens']['<|begin_of_text|>']['ids'] = [128000+idx]
    
    dump_dir = f'EBM-Filter-FromScratch-Llama3_Vocab/EBM_{fname.split("/")[-1][:-4]}'
    tokenizer_base.save_pretrained(dump_dir)
    
    with open(dump_dir+'/tokenizer.json', 'w',encoding='utf-8') as f:
        json.dump(tokenizer_json, f)
    f.close()

In [ ]:
import pandas as pd
df = pd.read_csv('../Llama-3-EBM-MedicalLookup-Fragment/EBM_SplitMoreThan1_OOV.csv')
freq_ebm = df['Count'].to_list()
terms_EBM = df['Word'].to_list()
split_bart = df['Splits'].to_list()

sum_num = 0.
sum_den = 0.
for idx,term in enumerate(terms_EBM):
    sum_num += split_bart[idx]*freq_ebm[idx]
    sum_den += freq_ebm[idx]

old_score = sum_num/sum_den


import glob
from collections import defaultdict

dict_scores = defaultdict(lambda : defaultdict(dict))
for fname in sorted(glob.glob('./EBM-Filter-FromScratch-Llama3_Vocab/*') ,key = lambda x: int(x.split('/')[-1].split('_')[-1][:-1])):
    print(fname)
    try:
        domain_tok = AutoTokenizer.from_pretrained(fname)
    except Exception as e:
        print('Error in loading:',fname, e)
        continue
    sum_num = 0.
    sum_den = 0.
    
    for idx,term in enumerate(terms_EBM):
        sum_num += min(len(domain_tok.tokenize(term)),len(domain_tok.tokenize(' '+term)))*freq_ebm[idx]
        sum_den += freq_ebm[idx]

    key = fname.split('/')[-1].split('_')[-1]
    dict_scores[key] = [round(sum_num/sum_den,2),len(domain_tok)-128256]

with open(f'EBM-FERTILITY','a') as f:
    f.write(f'-------------\nEBM-Filter-SelfFromScratch\n--------------------\n')
    f.write('LLM_Tok: '+str(round(old_score,2))+'\n')

    for k1 in dict_scores:
        f.write(k1+'\t')
        f.write(f'{dict_scores[k1][0]}/{dict_scores[k1][1]}\t')
        f.write('\n')
f.close()



In [ ]:
idx = 0
gain_in_fragments = []
tokenizer = AutoTokenizer.from_pretrained("./EBM-Filter-FromScratch-Llama3_Vocab/EBM_20K/")
import numpy as np
with open('../../../../../TxtInputFiles/PAC_input.txt') as f:
    for line in f:
        try:
            line = line.strip()
            org_enc = tokenizer_base.encode(line)
            vocab_enc = tokenizer.encode(line)
            
            org_dec = tokenizer_base.decode(org_enc)
            vocab_dec = tokenizer.decode(vocab_enc)
            
            assert org_dec == vocab_dec, f'Failed at {idx}'
            
            gain_in_fragments.append((len(org_enc)-len(vocab_enc))/len(org_enc))
            
            idx += 1
            if idx%1000 == 0:
                print(f'Processed {idx}.... {np.percentile(gain_in_fragments, [0,10,50,90,100])}')
        except:
            print(f'--------------------Failed at {idx}--------------------')
            print(f'Orig : {org_dec}')
            print(f'Vocab: {vocab_dec}')
    

In [ ]:
np.percentile(gain_in_fragments, [0,10,50,90,100])

In [ ]:
import torch
import math
from transformers import AutoModelForCausalLM, AutoTokenizer
target_tokenizer = AutoTokenizer.from_pretrained("./EBM-Filter-FromScratch-Llama3_Vocab/EBM_20K/")
source_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B")

In [ ]:
def round_to_nearest_multiple(vocabulary_size, multiple):
    rounded_size = math.ceil(vocabulary_size / multiple) * multiple
    return rounded_size

def instantiate_model_by_mean(
    source_model: AutoModelForCausalLM,
    source_tokenizer: AutoTokenizer,
    target_tokenizer: AutoTokenizer,
    tie_word_embeddings: bool = False
):
    # Determine the device (GPU or CPU)
    print("Inside instantiate model by mean")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # Initialize source embeddings
    source_embeddings = source_model.get_input_embeddings().weight#.detach().to(device)
    target_embeddings = torch.zeros(
        (round_to_nearest_multiple(len(target_tokenizer), 8), 
         source_embeddings.shape[1]), device=device
    )
    
    #PLM-Part
    target_embeddings[:128000] = source_embeddings[:128000]
    #Reserved Tokens
    for idx in range(target_tokenizer.bos_token_id,len(target_tokenizer)):
        target_embeddings[idx] = source_embeddings[source_tokenizer.convert_tokens_to_ids(target_tokenizer.convert_ids_to_tokens(idx))]

    if not tie_word_embeddings:
        print("You are using the output projection init.")
        source_head_embeddings = source_model.get_output_embeddings()#.weight.detach().to(device)
        target_head_embeddings = torch.zeros(
            (round_to_nearest_multiple(len(target_tokenizer), 8), 
             source_head_embeddings.shape[1]), device=device
        )
        
        #PLM-Part
        target_head_embeddings[:128000] = source_head_embeddings[:128000]
        #Reserved Tokens        
        for idx in range(target_tokenizer.bos_token_id,len(target_tokenizer)):
            target_head_embeddings[idx] = source_head_embeddings[source_tokenizer.convert_tokens_to_ids(target_tokenizer.convert_ids_to_tokens(idx))]

    # Initialize the rest of the embeddings
    for i in range(len(source_tokenizer)-256, len(target_tokenizer)-256):
        token = target_tokenizer.convert_ids_to_tokens(i)
        if token.startswith('Ġ'): token = ' '+token[1:]
        source_ids = source_tokenizer.convert_tokens_to_ids(source_tokenizer.tokenize(token))
        source_ids = torch.tensor(source_ids)#, device=device)
        target_embeddings[i] = source_embeddings[source_ids].mean(dim=0)
        print(i,token, target_embeddings[i], source_ids)
        if not tie_word_embeddings:
            target_head_embeddings[i] = source_head_embeddings[source_ids].mean(dim=0)

    # Expand the embeddings
    target_model = source_model #AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")
    target_model.resize_token_embeddings(
        len(target_tokenizer), 
        pad_to_multiple_of=8  # See https://github.com/huggingface/transformers/issues/26303
    )
    target_model.get_input_embeddings().weight.data = target_embeddings
    target_model.config.vocab_size = round_to_nearest_multiple(len(target_tokenizer), 8)
    
    if not tie_word_embeddings:
        target_model.get_output_embeddings().weight.data = target_head_embeddings

    return target_model, target_tokenizer

In [ ]:
source_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")
old_embeddings = source_model.get_input_embeddings().weight[source_tokenizer.bos_token_id:].detach().cpu().numpy().copy()



In [ ]:
old_embeddings

In [ ]:
target_model, target_tokenizer = instantiate_model_by_mean(source_model,source_tokenizer,target_tokenizer)

In [ ]:
len(target_tokenizer), len(source_tokenizer), target_tokenizer.bos_token_id, source_tokenizer.bos_token_id, len(source_model.get_input_embeddings().weight)

In [ ]:
new_embeddings = target_model.get_input_embeddings().weight[target_tokenizer.bos_token_id:len(target_tokenizer)].detach().cpu().numpy().copy()

In [ ]:
import numpy 
for v1,v2 in zip(old_embeddings,new_embeddings):
    # s_id = source_tokenizer.convert_tokens_to_ids(target_tokenizer.convert_ids_to_tokens(idx))
    assert numpy.all(v1 == v2), f'Failed at {v1},{v2}'
    # print(f'Passed at {idx}')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.1-8B')

vocab_tokens = [x.strip() for x in open('./EBM/20K.txt').read().splitlines()]

max_token, max_token_to_add = '',0
for word in vocab_tokens:
    tokens_to_add = len(tokenizer.tokenize(word if not word.startswith('Ġ') else ' '+word[1:]))-2
    if tokens_to_add > max_token_to_add:
        max_token = word
        max_token_to_add = tokens_to_add
        print(word,tokens_to_add)

In [ ]:
tokenizer.tokenize(' hypercholesterolemia')